In [1]:
import pandas as pd


In [6]:
df=pd.read_csv('fake_train.csv', encoding='latin1', on_bad_lines='skip', engine='python')

In [7]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didnât Even See Comeyâs...,Darrell Lucus,House Dem Aide: We Didnât Even See Comeyâs...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [8]:
df.isnull().sum()

,0
id,0
title,51
author,209
text,4
label,0


In [9]:
df.shape

(1975, 5)

In [11]:
df=df.dropna()

In [12]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didnât Even See Comeyâs...,Darrell Lucus,House Dem Aide: We Didnât Even See Comeyâs...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [13]:
X=df.drop('label',axis=1)

In [14]:
y=df['label']

In [15]:
y.value_counts()

,count
label,
0,966
1,749


In [16]:
import tensorflow as tf

In [17]:
tf.__version__

'2.19.0'

In [18]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional

In [19]:
voc_size=5000

In [20]:
messages=X.copy()

In [21]:
messages.head()

,id,title,author,text
0,0,House Dem Aide: We Didnât Even See Comeyâs...,Darrell Lucus,House Dem Aide: We Didnât Even See Comeyâs...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [22]:
import nltk
import re
from nltk.corpus import stopwords

In [23]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [25]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for idx in messages.index:
    print(idx)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][idx])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

0
1
2
3
4
5
7
9
10
11
12
13
14
15
16
17
18
19
21
22
24
25
26
27
28
29
30
32
33
34
35
36
37
38
39
40
41
42
44
45
46
48
49
50
52
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
97
98
99
100
101
102
103
104
105
106
108
109
110
111
112
113
114
115
116
117
118
119
121
122
123
125
126
127
128
129
130
131
132
133
136
137
139
141
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
171
172
173
174
176
177
178
180
181
182
183
184
185
187
188
189
190
191
192
193
194
197
199
200
201
202
203
205
206
207
208
210
211
212
213
214
216
217
218
219
220
221
222
224
225
226
227
228
230
232
233
234
237
238
239
240
241
242
243
244
245
246
247
248
249
251
252
253
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
284
285
286
287
289
290
291
293
294
295
296
297
298
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314


In [26]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [27]:
onehot_repr=[one_hot(words,voc_size) for words in corpus]
onehot_repr

[[2512, 3658, 2175, 1339, 114, 1662, 3254, 1132, 1345, 3904],
 [510, 852, 4153, 3560, 4104, 285, 1912],
 [807, 2560, 4293, 2725],
 [3033, 1972, 583, 2631, 2390, 4604],
 [3337, 4104, 4002, 3187, 21, 3405, 4104, 1968, 877, 1640],
 [3686,
  2961,
  3640,
  1432,
  2321,
  766,
  3834,
  3571,
  2130,
  2040,
  4085,
  3552,
  2564,
  4491,
  1912],
 [3687, 4676, 2317, 2765, 170, 3689, 255, 190, 618, 2751, 2631],
 [629, 1693, 704, 3343, 2953, 3356, 766, 632, 618, 2751, 2631],
 [4004, 731, 4240, 519, 2574, 1704, 14, 1772, 766, 2839],
 [3139, 4647, 3054, 578, 2890, 2038, 3324, 2501],
 [268, 613, 1433, 238, 4367, 1856, 3435, 4348, 1451, 25, 4306],
 [2631, 4792, 114, 1704, 766, 2953],
 [3687, 2920, 351, 18, 3912, 1937, 1686, 4826, 1317],
 [2631, 2600, 2568, 1950, 1099, 2679, 4268, 618, 2751, 2631],
 [2053, 69, 2603, 775, 1478, 618, 2751, 2631],
 [2059, 483, 1112, 2953, 2375, 2191, 3435, 2365, 1605, 3849],
 [1284, 2237, 852],
 [686, 4146, 4567, 4688, 766, 3563, 4978, 1912],
 [488, 3696, 4153, 7

In [31]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)

embedded_docs


array([[   0,    0,    0, ..., 1132, 1345, 3904],
       [   0,    0,    0, ..., 4104,  285, 1912],
       [   0,    0,    0, ..., 2560, 4293, 2725],
       ...,
       [   0,    0,    0, ...,  618, 2751, 2631],
       [   0,    0,    0, ...,  618, 2751, 2631],
       [   0,    0,    0, ..., 2053,  808, 3649]], dtype=int32)

In [34]:
from tensorflow.keras.layers import Input

embedding_vector_features = 40

model = Sequential()
model.add(Input(shape=(sent_length,)))   # 👈 builds the model
model.add(Embedding(voc_size, embedding_vector_features))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ (None, 20, 40)         │       200,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_3 (Bidirectional) │ (None, 200)            │       112,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           201 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 313,001 (1.19 MB)

 Trainable params: 313,001 (1.19 MB)

 Non-trainable params: 0 (0.00 B)

In [35]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [36]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [37]:
## Model Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=32)

Epoch 1/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 6s 21ms/step - accuracy: 0.5761 - loss: 0.6566 - val_accuracy: 0.7933 - val_loss: 0.3943
Epoch 2/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8919 - loss: 0.2584 - val_accuracy: 0.8958 - val_loss: 0.2852
Epoch 3/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9755 - loss: 0.0799 - val_accuracy: 0.8905 - val_loss: 0.3872
Epoch 4/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9911 - loss: 0.0206 - val_accuracy: 0.8922 - val_loss: 0.4644
Epoch 5/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 1.0000 - loss: 0.0027 - val_accuracy: 0.8816 - val_loss: 0.5580
Epoch 6/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 1.0000 - loss: 0.0013 - val_accuracy: 0.8852 - val_loss: 0.5965
Epoch 7/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 1.0000 - loss: 7.4464e-04 - val_accuracy: 0.8869 - val_loss: 0.6235
Epoch 8/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 1.0000 - loss: 5.3059e-04 - val_accuracy: 0.

In [38]:
y_pred=model.predict(X_test)

18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step


In [39]:
import numpy as np

y_pred=np.where(y_pred>=0.5,1,0)

In [40]:
from sklearn.metrics import confusion_matrix,accuracy_score
confusion_matrix(y_test,y_pred)

array([[291,  25],
       [ 40, 210]])

In [41]:
print(accuracy_score(y_test,y_pred))

0.8851590106007067
